# Задание 3

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import pydotplus

from sklearn import tree, cross_validation, metrics
from IPython.display import Image

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline


In [ ]:
def class